# 保留易观指数的原始数据


In [2]:
import requests
import json
import time
from pandas.io.json import json_normalize

def getDataFromNet():
    allInOne = []
    for index in range(1,11):
        url = 'http://zhishu.analysys.cn/public/qianfan/topRank/listTopRank?page=' + str(index) + '&pageSize=200'
        #print(url)
        r = requests.get(url)

        dataStr = r.text
        data = json.loads(dataStr)

        data_list = data['datas']['list']
        allInOne = allInOne + data_list
        
    df = json_normalize(allInOne)
    #处理初始数据，添加业务关心的数据
    df['dayChangeNums'] = df['dayActiveNumsRatio'] * df['dayActiveNums']
    df['monthChangeNums'] = df['monthActiveNumsRatio'] * df['monthActiveNums']
    
    return df
    
    
def write2File(df):
    fileName = 'data_20190226.csv'
    df.to_csv(fileName, index=False, encoding = 'UTF-8')
    time.sleep(2)


df = getDataFromNet()
write2File(df)
print('done')

done


## 用来处理广告来源数据的分析

* 通过url的后缀进行分割，添加一列关键技术点（或者是分享）
* 统计文章的时间，来分析用户是博客订阅者，还是通过搜索引擎过来的
* 针对访问超过一次的用户，单独进行分析

In [3]:
import pandas as pd
archive_origin = pd.read_csv('ryf_source.csv', encoding = 'utf8')
ad_df = archive_origin.copy()

ad_df.head()

,Unnamed: 0,visit_date,domain,ref_link,portal_page,keywords,visit_IP,unique_no,total_visit_duration_time,total_visit_page_one_time,...,brewer,lang,screen,screen_color,Flash,has_cookie,has_java,this_time_visit_start,visit_portal_time,portal_url
0,1063.0,2019/02/22 13:42:29,深圳,360搜索,http://www.youkeda.com,--,219.133.66.223,1839110690557959428,14s,1.0,...,Google Chrome,NaN,1920x1080,24-bit,NaN,支持,支持,2019-02-22 13:42:29,14s,http://www.youkeda.com
1,187.0,2019/02/27 14:42:52,杭州,百度,https://www.bmatch.tech,bmatch,115.192.184.82,1885452069639056133,307s,3.0,...,Google Chrome,NaN,1680x1050,24-bit,NaN,支持,不支持,2019-02-27 14:42:52,194s,https://www.bmatch.tech
2,906.0,2019/02/25 12:25:09,杭州,百度,https://www.bmatch.tech,b.match,112.17.240.209,2078682566481140752,15s,3.0,...,Safari移动版,NaN,NaN,32-bit,NaN,支持,不支持,2019-02-25 12:25:09,12s,https://www.bmatch.tech
3,1022.0,2019/02/22 21:19:22,杭州,百度,https://www.bmatch.tech,网易云课堂,60.186.223.214,1917635642669449575,38s,3.0,...,Chrome移动版,NaN,NaN,32-bit,NaN,支持,不支持,2019-02-22 21:19:22,25s,https://www.bmatch.tech
4,186.0,2019/02/27 14:47:44,杭州,百度,https://www.bmatch.tech,bmatch,36.27.67.31,2294947012308152234,98s,1.0,...,Google Chrome,NaN,1680x1050,24-bit,NaN,支持,不支持,2019-02-27 14:47:44,98s,https://www.bmatch.tech


# 数据清洗

* 只分析来源是阮一峰广告的记录
* 过滤掉公司IP
* 两个停留时间进行处理

In [4]:
#入口请求，阮一峰的页面
ryf_url = 'https://www.bmatch.tech/course/react?from=ryf'
buy_ad_url = 'http://www.ruanyifeng.com/support.html'
tongji_url = 'https://tongji.baidu.com'
#公司ip
company_ip = '36.27.67.31'

ad_df = ad_df[(ad_df['portal_page'] == ryf_url) & (ad_df['visit_IP'] != company_ip)]
ad_df = ad_df[ad_df['ref_link'] != buy_ad_url]

def time_str_to_second(str):
    second = 0
    if str[0].isdigit():
        time_str = str[:-1]
        second = int(time_str)
    return second  

def handle_str(row):
    row['total_visit_duration_time'] = time_str_to_second(row['total_visit_duration_time'])
    row['visit_portal_time'] = time_str_to_second(row['visit_portal_time'])

    return row

ad_df = ad_df.apply(handle_str, axis = 1)


## 从URL中获取文章的标题和更新时间信息

返回的数据结构如下： <br>
`{
   'title':'react',
   'date':'2019/02'
}`

In [6]:
from urllib.parse import urlparse

def getTitle(url):
    if url.startswith('http'):
        urlobj = urlparse(url)
        # 将url用“/”进行分割，同时去掉.html尾缀
        str_list = urlobj.path.split('/')
        if 'blog' in str_list:
            seq = (str_list[2], str_list[3])
            date = '@'.join(seq)
            title = urlobj.path.split('/')[-1][:-5]
            title = date + '@' + title
            return title
    return ''

In [7]:
# 从文件夹中获取已经存在的文件名称的list
import os
html_file_dir = 'ryf_html'

existed_files = []
for file in os.listdir(html_file_dir):
    if os.path.splitext(file)[1] == '.html':
        existed_files.append(os.path.splitext(file)[0])

#Index的nparray 转换成list
all_blog_files = ad_df['ref_link'].map(getTitle).drop_duplicates().values.tolist()


download_files = list(set(all_blog_files).difference(set(existed_files)))
download_files

['',
 '2011@05@how_to_choose_free_software_licenses',
 '2008@12@a_puzzle_from_terrence_tao',
 '2015@02@mvcmvp_mvvm',
 '2017@03@gartner-hype-cycle',
 '2017@02@filename-should-be-lowercase',
 '2018@02@docker-wordpress-tutorial',
 '2019@02@npx',
 '2011@12@ssh_port_forwarding',
 '2011@09@restful',
 '2017@08@koa',
 '2007@03@metadata',
 '2016@08@boolean-algebra',
 '2015@03@react',
 '2012@04@functional_programming',
 '2012@06@sass',
 '2009@08@learning_javascript_closures',
 '2011@08@jquery_best_practices',
 '2014@10@event-loop',
 '2016@06@dns',
 '2018@11@most-effective-sales-promotion',
 '2018@10@runner',
 '2015@09@matrix-multiplication',
 '2018@07@my-books',
 '2019@02@weekly-issue-42',
 '2019@01@prolog',
 '2017@07@working-poor',
 '2011@04@quicksort_in_javascript',
 '2016@09@react-technology-stack',
 '2015@12@git-workflow',
 '2013@12@getting_started_with_postgresql',
 '2014@05@oauth_2_0',
 '2017@05@websocket',
 '2016@02@youth-by-coetzee',
 '2016@11@javascript',
 '2009@01@what_is_a_circle',
 '

In [8]:
import requests
def filename2url(name):
    url = ""
    if name.strip() and name:
        url = 'http://www.ruanyifeng.com/blog/' + name.replace("@","/") + '.html';
    return url;

def download_html(file):
    url = filename2url(file)
    if url.strip() and url:  
        response = requests.get(url)
        file_name = '@'.join(url.split('/')[-3:])
        with open(os.path.join(html_file_dir, file_name), mode='wb') as file:
            file.write(response.content)
        return 

for name in download_files:
    download_html(name)
    
print('download done')

download done


# 从文章的HTML结构中获取信息

* 文章的类目从HTML结构中获取
* 判断分类是否属于意向用户的范围（开发者手册，以及JavaScript偏向前端开发角色）<br>

返回的数据结构如下：<br>
`
{
  'intention': False/True,
  'category':'JavaScript'
}
`

In [9]:
from bs4 import BeautifulSoup
good_category = ['开发者手册','JavaScript']

def getCategory(url):
    data = {'intention': False, 'category':''}

    if 'www.' in url:
        urlobj = urlparse(url)
        file_name = '@'.join(urlobj.path.split('/')[-3:])
        file_name = html_file_dir + '/' + file_name
        with open(file_name, 'r') as f:
            content = f.read()
            soup = BeautifulSoup(content, 'lxml')
            categorySpan = soup.find('div', {'class': 'entry-categories'})
            if not categorySpan:
                return data
            category = categorySpan.find('a').string
            if category in good_category:
                data['intention'] = True
            data['category'] = category
            return data
    return data

def getTitleInfo(url):
    if 'www.' in url:
        urlobj = urlparse(url)
        # 将url用“/”进行分割，同时去掉.html尾缀
        str_list = urlobj.path.split('/')
        if 'blog' in str_list:
            seq = (str_list[2], str_list[3])
            date = '/'.join(seq)
            title = urlobj.path.split('/')[-1][:-5]
            return {'title': title, 'date':date}
    return {'title': '', 'date':''}


# 得到处理后的文章标题，文章写作时间，是否是意向用户，以及文章的分类
def analysis(url):
    data = {'intention': False, 
            'category':'',
            'title':'',
            'article_written':'',
            'ref_link':url}

    
    title_info = getTitleInfo(url)
    data['article_written'] = title_info['date']
    data['title'] = title_info['title']
    html_info = getCategory(url)
    data['category'] = html_info['category']
    data['intention'] = html_info['intention']
    return data

In [10]:
# 测试代码
url_list = ad_df['ref_link'].drop_duplicates()[:2]
print(url_list)
for url in url_list:
    data = analysis(url)
    print(data)

905                                                 直接访问
962    http://www.ruanyifeng.com/blog/2006/02/post_17...
Name: ref_link, dtype: object
{'intention': False, 'category': '', 'title': '', 'article_written': '', 'ref_link': '直接访问'}
{'intention': False, 'category': '观点与感想', 'title': 'post_179', 'article_written': '2006/02', 'ref_link': 'http://www.ruanyifeng.com/blog/2006/02/post_179.html'}


# 得到处理后的数据表格，和原表格进行合并


In [11]:
url_list = ad_df['ref_link'].drop_duplicates()
df_list = []

for url in url_list :
    data = analysis(url)
    df_list.append(data)


#将两个数据结构按照同一个key值进行合并
new_df = pd.DataFrame(df_list, columns = ['ref_link', 'title', 'category', 'article_written', 'intention'])
new_ad_df = pd.merge(ad_df, new_df, how='inner', on=['ref_link'])
new_ad_df.head()

#将得到的新表格，写入到本地文件
new_ad_df.to_csv('handled_ryf_data.csv', index=False, encoding = 'utf8')